In [11]:
from result_saver import SaverProvider
provider = SaverProvider()

In [28]:
import numpy as np
# parameter grid
distance = 30
nb_of_rounds = 30
device = "ibm_torino"
logicals = ['0', '1']

In [29]:
import pickle
with open('longest_path_torino.pkl', 'rb') as f:
    path = pickle.load(f)
print(len(path)//2)

56


In [30]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import find_longest_path_in_hex
from Scratch import metadata_helper


backend = provider.get_backend(device)

bounded_path = path[:2 * distance - 1]
layout = bounded_path[1::2] + bounded_path[::2] 
code = RepetitionCodeCircuit(distance, nb_of_rounds) 
for logical in logicals:          
    qc = code.circuit[logical]                                   
    transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=0)

    n_shots = int(1e6 / (distance*nb_of_rounds) / 10)   # for torino 
    # n_shots = 1
    metadata = metadata_helper(descr='diff rounds v2', code="RepetitionCodeCircuit", distance=int(distance), rounds=f"{nb_of_rounds}",  logical=logical, layout='_is_hex=True, sherbrooke')
    backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', job_tags=[f"RC({distance}, {nb_of_rounds}), {n_shots}"]) # meas_level=1 for IQ  
    # print(transpiled_qc.draw(fold=-1))
    